# Parsing SEC Filing Document


## Objective

Parse the filing TXT file (because XBRL is not available for all filings for the years before the XBRL is mandated) to create a DOM like structure that represent the filing data

## References

* [YouTube Parsing SEC Filings (Newer Ones) in Python | Part 1](https://www.youtube.com/watch?v=TxUmufNnIaA) - Overview and SEC header.
* [Parsing SEC Filings (Newer Ones) in Python | Part 2](https://www.youtube.com/watch?v=Wi9PPoS9cxI)
* [Parsing SEC Filings (Newer Ones) in Python | Part 3](https://www.youtube.com/watch?v=ak_1TG_kuDU&t=0s)
* [Github Web Scraping SEC - Parsing SEC Documents - New Filings.ipynb](https://github.com/areed1192/sigma_coding_youtube/blob/master/python/python-finance/sec-web-scraping/Web%20Scraping%20SEC%20-%20Parsing%20SEC%20Documents%20-%20New%20Filings.ipynb)

> In this notebook, we will go over how to parse SEC filing documents so you can extract different content from it and also logically organize the information. 


# EDGAR Filing TXT 

```.txt``` file is all-in-one **XML-ish** (because some tags are not closed e.g. TYPE, SEQUENCE) file enclosing all the data of a specific filing identified with an **accession number**. 


* [0001193125-14-277160.txt](https://www.sec.gov/Archives/edgar/data/320193/000119312514277160/0001193125-14-277160.txt)

```
<SEC-DOCUMENT>0001193125-14-277160.txt : 20140723
    <SEC-HEADER>0001193125-14-277160.hdr.sgml : 20140723
        <ACCEPTANCE-DATETIME>20140723163248
            ACCESSION NUMBER:       0001193125-14-277160
            CONFORMED SUBMISSION TYPE:  10-Q
            PUBLIC DOCUMENT COUNT:      10
            CONFORMED PERIOD OF REPORT: 20140628
            FILED AS OF DATE:       20140723
            DATE AS OF CHANGE:      20140723
 
            FILER:
 
            COMPANY DATA:
            COMPANY CONFORMED NAME:         APPLE INC
            CENTRAL INDEX KEY:          0000320193
            STANDARD INDUSTRIAL CLASSIFICATION: ELECTRONIC COMPUTERS [3571]
            IRS NUMBER:             942404110
            STATE OF INCORPORATION:         CA
            FISCAL YEAR END:            0927
 
            FILING VALUES:
            FORM TYPE:      10-Q
            SEC ACT:        1934 Act
            SEC FILE NUMBER:    000-10030
            FILM NUMBER:        14989142
 
            BUSINESS ADDRESS:
            STREET 1:       ONE INFINITE LOOP
            CITY:           CUPERTINO
            STATE:          CA
            ZIP:            95014
            BUSINESS PHONE:     (408) 996-1010
 
            MAIL ADDRESS:
            STREET 1:       ONE INFINITE LOOP
            CITY:           CUPERTINO
            STATE:          CA
            ZIP:            95014
 
            FORMER COMPANY:
            FORMER CONFORMED NAME:  APPLE COMPUTER INC
            DATE OF NAME CHANGE:    19970808
    </SEC-HEADER>
    <DOCUMENT>
        <TYPE>10-Q
        <SEQUENCE>1
        <FILENAME>d740164d10q.htm
        <DESCRIPTION>10-Q
        <TEXT>
            <HTML><HEAD>                            ...
                <TITLE>10-Q</TITLE>
            </HEAD>
            <BODY BGCOLOR="WHITE">
            </BODY></HTML>
        </TEXT>
    </DOCUMENT>
... (repeats of DOCUMENT)   
</SEC-DOCUMENT>
```

## DOCUMENT

Each DOCUMENT corresponds to a html file identified with FILENAME (d740164d10q.htm) and the HTML tag has its content.

```
    <DOCUMENT>
        <TYPE>10-Q                              # <--- No closing tag
        <SEQUENCE>1
        <FILENAME>d740164d10q.htm
        <DESCRIPTION>10-Q
        <TEXT>
            <HTML><HEAD>                        # <--- <HTML> has closing </HTML> tag
                <TITLE>10-Q</TITLE>
            </HEAD>
            <BODY BGCOLOR="WHITE">
            </BODY></HTML>
        </TEXT>
    </DOCUMENT>
```


### DOCUMENT Elements

* TYPE : identifies the type of the file e.g. 10-K.
* SEQAUENCE
* FILENAME
* DESCRIPTION
* TEXT/HTML



# Beautifulsoup4
Beautiful Soup is a Python package for parsing HTML and XML documents. It creates a parse tree for parsed pages.

## BS4 LXML Parser

* Fast.
* Be able to handle non-closing tag, e.g. ```<TYPE>10-Q```
* Drops ```<HTML>```, ```</HTML>``` tags.

## BS4 HTML Parser

* Python built-in parser.
* Be able to handle non-closing tag, e.g. ```<TYPE>10-Q```
* Preserve  ```<HTML>```, ```</HTML>``` tags.

---
# Setup

In [1]:
# define the url to specific html_text file
FILING_DIR_URL = "https://www.sec.gov/Archives/edgar/data/320193/000119312514277160"
TEXT_NAME = "0001193125-14-277160.txt"
TEXT_URL = "/".join([FILING_DIR_URL, TEXT_NAME])

TEXT_URL

'https://www.sec.gov/Archives/edgar/data/320193/000119312514277160/0001193125-14-277160.txt'

In [2]:
!pip install lxml

In [3]:
import re
import requests
import unicodedata
from bs4 import BeautifulSoup
from IPython.core.display import (
    display, 
    HTML
)

In [4]:
%%html
<style>
table {float:left}
</style>

In [5]:
def restore_windows_1252_characters(restore_string):
    """
        Replace C1 control characters in the Unicode string s by the
        characters at the corresponding code points in Windows-1252,
        where possible.
    """

    def to_windows_1252(match):
        try:
            return bytes([ord(match.group(0))]).decode('windows-1252')
        except UnicodeDecodeError:
            # No character at the corresponding code point: remove it.
            return ''
        
    return re.sub(r'[\u0080-\u0099]', to_windows_1252, restore_string)

---
# Load EDGAR Filing TXT 

Download the ```.txt``` file from EDGAR. SEC now requires user-agent header.

In [6]:
headers = {"User-Agent": "Company Name myname@company.com"}
response = requests.get(TEXT_URL, headers=headers)

if response.status_code == 200:
    content = response.content.decode("utf-8") 
else:
    print(f"{TEXT_URL} failed with status {response.status_code}")

---
## BS4 with HTML Parser

Use HTML parser as it can handle non closing tags and preverve the HTML tag.


In [7]:
soup = BeautifulSoup(content, 'html.parser')

---
# Filing DOM

Create a DOM like structure that represent the filing data using Python dictionnary. Each filing (identified by the accession number) creates a dictionary hierarchy  in the DOM by paring the TXT file.

In [8]:
# define a dictionary that will house all filings.
master_filings_dict = {}

# let's use the accession number as the key. This 
accession_number = '0001193125-14-277160'

# add a new level to our master_filing_dict, this will also be a dictionary.
master_filings_dict[accession_number] = {}

# this dictionary will contain two keys, the sec header content, and a documents key.
master_filings_dict[accession_number]['sec_header_content'] = {}
master_filings_dict[accession_number]['filing_documents'] = None

## SEC-HEADER

Extract the SEC-HEADER element from the filing.

In [9]:
sec_header_tag = soup.find('sec-header')
master_filings_dict[accession_number]['sec_header_content']['sec_header_code'] = sec_header_tag
display(sec_header_tag)

<sec-header>0001193125-14-277160.hdr.sgml : 20140723
<acceptance-datetime>20140723163248
ACCESSION NUMBER:		0001193125-14-277160
CONFORMED SUBMISSION TYPE:	10-Q
PUBLIC DOCUMENT COUNT:		10
CONFORMED PERIOD OF REPORT:	20140628
FILED AS OF DATE:		20140723
DATE AS OF CHANGE:		20140723

FILER:

	COMPANY DATA:	
		COMPANY CONFORMED NAME:			APPLE INC
		CENTRAL INDEX KEY:			0000320193
		STANDARD INDUSTRIAL CLASSIFICATION:	ELECTRONIC COMPUTERS [3571]
		IRS NUMBER:				942404110
		STATE OF INCORPORATION:			CA
		FISCAL YEAR END:			0927

	FILING VALUES:
		FORM TYPE:		10-Q
		SEC ACT:		1934 Act
		SEC FILE NUMBER:	000-10030
		FILM NUMBER:		14989142

	BUSINESS ADDRESS:	
		STREET 1:		ONE INFINITE LOOP
		CITY:			CUPERTINO
		STATE:			CA
		ZIP:			95014
		BUSINESS PHONE:		(408) 996-1010

	MAIL ADDRESS:	
		STREET 1:		ONE INFINITE LOOP
		CITY:			CUPERTINO
		STATE:			CA
		ZIP:			95014

	FORMER COMPANY:	
		FORMER CONFORMED NAME:	APPLE COMPUTER INC
		DATE OF NAME CHANGE:	19970808
</acceptance-datetime></sec-heade

## DOCUMENT

Generate a DOM for DOCUMWENT element.

```
<DOCUMENT>
    <TYPE>10-Q
    <SEQUENCE>1
    <FILENAME>d740164d10q.htm
    <DESCRIPTION>10-Q
    <TEXT>
        <HTML>
            <HEAD><TITLE>${DOCUMEHT_TYPE}</TITLE></HEAD>
            <BODY BGCOLOR="WHITE">...</BODY>
        </HTML>
    </TEXT>
</DOCUMENT>
```

In [10]:
def split_document_html_into_pages(html: str):
    """
    Extract each page in the HTML tag using the HR tag string as the page break.
    HTML in DOCUMENT is sectioned into pages with the HR tag, e.g. 
    <hr align="CENTER" size="3" style="COLOR:#999999" width="100%"/>. 
    
    Args:
        HTML in string format, not BS4 tag object.
    """
    assert isinstance(html, str), f"Invalid arg of type {type(html)}"
    
    page_breaks_regexp = "|".join([
        re.escape(str(page_break)) 
        for page_break in BeautifulSoup(html, 'html.parser').find_all('hr', {'width':'100%'})
    ])
    pages = re.split(page_breaks_regexp, str(html))
    return pages


def get_document_dictionary(filing_document):
    """Extract the elements in the DOCUMENT tag."""
    document_dict = {}

    document_type = filing_document.type.find(text=True, recursive=False).strip()
    document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
    document_filename = filing_document.filename.find(text=True, recursive=False).strip()
    document_description = filing_document.description.find(text=True, recursive=False).strip()

    document_dict["type"] = document_type
    document_dict['sequence'] = document_sequence
    document_dict['filename'] = document_filename
    document_dict['description'] = document_description  

    # --------------------------------------------------------------------------------
    # Extract the HTML tag only. Note that it is bs4.element.Tag object, NOT string.
    # --------------------------------------------------------------------------------
    document_dict['html'] = filing_document.find('html').extract()
    document_dict['pages'] = split_document_html_into_pages(str(document_dict['html'].prettify()))
    
    return document_dict

Parse through the DOCUMENT elements

In [11]:
def get_document_dictionary_from_txt(soup):
    for filing_document in soup.find_all('document'):
        yield get_document_dictionary(filing_document)

## Elements in DOCUMENT

Examine the elements in the first DOCUMENT tag.

In [12]:
generator = get_document_dictionary_from_txt(soup)
document = next(generator)

### TYPE

In [13]:
document['type']

'10-Q'

### Sequence

In [14]:
document['sequence']

'1'

### Description

In [15]:
document['filename']

'd740164d10q.htm'

### HTML

In [16]:
# NOTE: HTML needs "str" for the data argument, cannot accept bs4.element.Tag
HTML(data=str(document['html']))

## Pages in HTML

Extract each page in the HTML. HTML in DOCUMENT is sectioned into pages with the ```HR``` tag, e.g. ```<hr align="CENTER" size="3" style="COLOR:#999999" width="100%"/>```.  The idea is using the ```HR``` tag string as the page break to split HTML into pages.

<img src="../image/edgar_filing_text_pages.png" align="left"/>

In [17]:
pages = document['pages']

### First page

In [18]:
if len(pages) > 0:
    display(HTML(data=pages[0]))
else:
    print("There is no page in HTML.")

### Third page

In [19]:
if len(pages) > 2:
    display(HTML(data=pages[2]))
else:
    print("There is no thrid page in HTML.")

In [22]:
print(pages[2])


    <h5 align="left">
     <a href="#toc">
      Table of Contents
     </a>
    </h5>
    <p style="margin-top:0px;margin-bottom:0px">
     <font size="2" style="font-family:Times New Roman">
      <b>
       <a name="tx740164_1">
       </a>
       PART I—FINANCIAL INFORMATION
      </b>
     </font>
    </p>
    <p style="font-size:6px;margin-top:0px;margin-bottom:0px">
    </p>
    <table border="0" cellpadding="0" cellspacing="0" style="BORDER-COLLAPSE:COLLAPSE" width="100%">
     <tr>
      <td align="left" valign="top" width="9%">
       <font size="2" style="font-family:Times New Roman">
        <b>
         <a name="tx740164_2">
         </a>
         Item 1.
        </b>
       </font>
      </td>
      <td align="left" valign="top">
       <font size="2" style="font-family:Times New Roman">
        <b>
         Financial Statements.
        </b>
       </font>
      </td>
     </tr>
    </table>
    <p align="center" style="margin-top:12px;margin-bottom:0px">
     <font siz